In [45]:
import requests
from bs4 import BeautifulSoup
import openai
import json
import pandas as pd
import pickle

import time
import re
#####################
from tokens.key import github_key # import your key
from tokens.key import openai_key # import your key

In [ ]:
RQ1_dict = {"Inefficient coding for target micro-architecure": {
        "Count": 58,
        "Percentage": 31.18,
        "Sub-Category": {
            "Memory/Data locality": {
                "Count": 36,
                "Percentage": 19.35,
                "Sub-Sub-Category": {
                    "Intra-thread data locality": {
                        "Count": 32,
                        "Percentage": 17.2,
                        "Sub-Sub-Sub-Category": "cache locality",
                    },
                    "Inter-thread Data locality": {
                        "Count": 4,
                        "Percentage": 2.15,
                        "Sub-Sub-Sub-Category": "cache locality",
                    },
                },
            },
            "Micro-architectural inefficiency": {
                "Count": 22,
                "Percentage": 11.83,
                "Sub-Sub-Category": {
                    "Missed compiler loop optimization": {
                        "Count": 9,
                        "Percentage": 4.84,
                        "Sub-Sub-Sub-Category": [
                            "missed unrolling",
                            "unnecessary branch",
                            "missed function inlining",
                        ],
                    },
                    "Loop carried dependence": {"Count": 1},
                    "suboptimal computational kernel size for the target": {
                        "Count": 6,
                        "Percentage": 3.23,
                        "Sub-Sub-Sub-Category": [
                            "Inefficient SIMD width for Power 10",
                            "Inefficient SIMD width for AMD Ryzen",
                            "Improper value for one parameter",
                            "Improper parameter values for Power10",
                            "Improper parameter values for Power10",
                            "Improper parameter values for SKYLAKEX CPUs",
                        ],
                    },
                    "Use of slow instruction set": {"Count": 3},
                    "suboptimal API for the target architecture": {
                        "Count": 3,
                        "Sub-Sub-Sub-Category": "Improper value for greedy parameter",
                    },
                },
            },
        },
    },
    "Missing parallelism": {
        "Count": 12,
        "Percentage": 6.45,
        "Sub-Category": {
            "Vector/SIMD parallelism": {"Count": 5, "Percentage": 2.69},
            "GPU parallelism": {"Count": 2},
            "Instruction level parallelism": {"Count": 1},
            "Task parallelism": {"Count": 4},
        },
    },
    "Parallelization overhead/inefficiency": {
        "Count": 15,
        "Percentage": 8.06,
        "Sub-Category": {
            "small parallel region": {"Count": 5},
            "Inefficeint thread mapping / inefficient block size / Load imbalance": {
                "Count": 2
            },
            "inefficient thread mapping": {"Count": 1},
            "Under-parallelization": {"Count": 2},
            "Over-Parallelization": {"Count": 2},
        },
    },
    "Inefficient Concurrency control and synchronization": {
        "Count": 7,
        "Percentage": 3.76,
        "Sub-Category": {
            "Unncessary locks": {"Count": 3},
            "coarse-grain locks": {},
            "Unncessary strong memory consistency": {"Count": 1},
            "Lock management overhead": {"Count": 1},
            "Unnecessary synchronization": {"Count": 2},
        },
    },
    "Unnecessary process communication": {
"Count": 2,
},
"Inefficient algorithm /data-structure and their implementation": {
"Count": 73,
"Percentage": 39.25,
"Sub-Category": {
"Unnecessary operation/traversal/function call": {
"Count": 13,
"Percentage": 6.99,
"Sub-Sub-Category": {
"Unnecessary computation": {"Count": 2},
"Unnecessary function call": {"Count": 1},
"Inefficient sparse array computation": {"Count": 1},
"Unnecessary traversal": {
"Count": 8,
"Percentage": 4.3,
"Sub-Sub-Sub-Category": "unnecessary sort",
},
"Unnecessary itermediate variable usage": {"Count": 1},
},
},
"Redundant operation": {
"Count": 16,
"Percentage": 8.6,
"Sub-Sub-Category": {
"Redundant traversal": {"Count": 1},
"Redundant allocation": {"Count": 2},
"Redundant computation": {
"Count": 12,
"Percentage": 6.45,
"Sub-Sub-Sub-Category": [
"multtranspose in loop",
"Dead store",
"Redundant sort operation",
"Redundant variable assignment operation",
"Redundant list splitting due to integer overflow",
],
},
"Unnecessary branching (??)": {
"Count": 1,
"Sub-Sub-Sub-Category": "Removing branch in function (branch optimization)",
},
},
},
"Expensive operation": {
"Count": 29,
"Percentage": 15.59,
"Sub-Sub-Category": {
"Expensive algorithm": {"Count": 3},
"Expensive runtime evaluation": {"Count": 3},
"Expensive many memory references": {
"Count": 3,
"Sub-Sub-Sub-Category": "Expensive data-structure traversal",
},
"Expensive atomic call": {
"Count": 1,
"Sub-Sub-Sub-Category": "API miss use",
},
"Expensive computational kernel for a target architecture": {"Count": 3},
"expensive high precision calculation": {"Count": 3},
"Expensive exponential operation(O(n) overhead)": {"Count": 1},
"Expensive tensor calculation": {"Count": 1},
"Expensive type-casting": {"Count": 2},
"Expensive library function call": {"Count": 2},
"Expensive integer division": {"Count": 1},
"Expensive function call": {"Count": 5},
"Expensive kernel operation for small matrix": {"Count": 1},
},
},
"Frequent function call": {"Count": 3},
"Inefficient data-structure library": {
"Count": 9,
"Percentage": 4.84,
"Sub-Sub-Category": {
"Slower data structure": {"Count": 2},
"Inefficient library API call": {
"Count": 7,
"Sub-Sub-Sub-Category": [
"dynamic contiguous array allocation when size known statically",
"Inefficient due to reference counting overhead",
],
},
},
},
"Usage of improper data type": {
"Count": 3,
"Sub-Sub-Category": { 
"Slowdown due to wrong data type": {
"Count": 3,
"Sub-Sub-Sub-Category": "CUDA kernel performance degradation due to data type",
},
},
},
},
},
"Inefficient memory management": {
"Count": 13,
"Percentage": 6.99,
"Sub-Category": {
"memory leak": {"Count": 5},
"repreated memory allocation": {"Count": 4},
"Redundant memory allocation": {"Count": 1},
"Slower memory allocation library call": {"Count": 1},
"Insufficient memory": {
"Count": 1,
"Sub-Sub-Category": "Inefficiency while computing large multiplication due to lower memory size",
},
"unnecessary data copy": {"Count": 1},
},
},
"I/O inefficiency": {
"Count": 2,
"Sub-Category": {
"sequential I/O operation": {"Count": 1},
"over parallelization": {"Count": 1},
},
},
"Unintentional Programming logic error": {
"Count": 3,
"Sub-Category": {
"boundary condition check": {"Count": 2},
},
},
"Inefficiency due to new compiler version": {
"Count": 1,
},
}


In [3]:
for i in RQ1_dict:
    print(i)

Inefficient coding for target micro-architecure
Missing parallelism
Parallelization overhead/inefficiency
Inefficient Concurrency control and synchronization
Unnecessary process communication
Inefficient algorithm /data-structure and their implementation
Inefficient memory management
I/O inefficiency
Unintentional Programming logic error
Inefficiency due to new compiler version


In [4]:
catogary_lsit_RQ2=[ 
"Domain and architecture agnostic algorithm and data structure optimization" ,
"Domain specific" ,
"Micro-architecture specific optimization" ,
"Enable compiler optimizations" ,
"Faster API call" ,
"Introduce parallelization" ,
"Balancing parallel load" ,
"Improved concurrency control" ,
"Memory management" ,
"fix programming error" ,]


In [5]:
sub_catogary_dict_RQ2 = {
    "Domain and architecture agnostic algorithm and data structure optimization": {
        "score": 17,
        "subcategories": [
            "data structure transformation",
            "lower complexity algorithm - domain and arch agnostic",
            "Caching",
            "Use of fast data strcture container",
        ],
    },
    "Domain specific": {
        "score": 27,
        "subcategories": [
            "Reduce iteration/traversal of data structure",
            "Eliminating unnecessary operation",
            "Avoid unnecessary communication",
            "Batching function calls",
            "lazy housekeeping",
            "code specialization",
            "lower complexity algorithm - domain specific",
        ],
    },
    "Micro-architecture specific optimization": {
        "score": 60,
        "subcategories": [
            "Architecture specific logic modification",
            "Architecture specific fast kernel",
            "reduced precision",
            "strength reduction",
            "Architecture specific fast instruction",
            "Use of data types that reduces computation and memory overhead",
            "Data locality optimization",
        ],
    },
    "Enable compiler optimizations": {
        "score": 27,
        "subcategories": [
            "Function inlining",
            "Enable compile time evaluation",
            "Scalar Replacement",
            "Loop optimization",
        ],
    },
    "Faster API call": {"score": 2, "subcategories": []},
    "Introduce parallelization": {
        "score": 11,
        "subcategories": [
            "OpenMP",
            "oneAPI",
            "Introduce vectorization",
            "pthread-based paralleism",
        ],
    },
    "Balancing parallel load": {
        "score": 17,
        "subcategories": [
            "Explicitly set thread numbers",
            "Sort workload and schedule",
            "Tune task size",
            "Reduce parallelization",
        ],
    },
    "Improved concurrency control": {
        "score": 7,
        "subcategories": [
            "Eliminate unnecessary lock",
            "Efficient lock implementation with atomics",
            "Replace strong consistency with an weaker one",
            "Eliminate unncessary synchronization",
            "Rewrite kernel to make memory access warp-synchronous",
        ],
    },
    "Memory management": {
        "score": 13,
        "subcategories": [
            "Avoid memory leak by memory management",
            "Pre-allocation",
            "Increase memory size for efficiency",
            "Use efficient API",
            "Reference rather than copy",
        ],
    },
    "fix programming error": {"score": 4, "subcategories": []},
    "total": 185,
}


In [6]:
sub_sub_catogary_dict_RQ2 = {
    "Domain and architecture agnostic algorithm and data structure optimization": {
        "count": 17,
        "subcategories": {
            "data structure transformation": 3,
            "lower complexity algorithm - domain and arch agnostic": 1,
            "Caching": 6,
            "Use of fast data strcture container": 7,
        },
    },
    "Domain specific": {
        "count": 27,
        "subcategories": {
            "Reduce iteration/traversal of data structure": 8,
            "Eliminating unnecessary operation": 8,
            "Avoid unnecessary communication": 2,
            "Batching function calls": 3,
            "lazy housekeeping": 1,
            "code specialization": 2,
            "lower complexity algorithm - domain specific": 3,
        },
    },
    "Micro-architecture specific optimization": {
        "count": 60,
        "subcategories": {
            "Architecture specific logic modification": 3,
            "Architecture specific fast kernel": 3,
            "reduced precision": 3,
            "strength reduction": 6,
            "Architecture specific fast instruction": 3,
            "Use of data types that reduces computation and memory overhead": 4,
            "Data locality optimization": {
                "count": 38,
                "sub_subcategories": [
                    "Avoid memory dereference by storing in register",
                    "Reorder memory reference to improve temporal locality",
                    "Reorder memory reference to improve spatial locality",
                    "Improve register/cache utilization via blocking",
                    "Avoid using GPU texture memory",
                    "Reduce GPU-global memory access",
                    "Thread-aware data access",
                    "thread scheduling to improve locality",
                    "Configure HW caches size",
                    "memory pinning",
                    "Memory prefetching",
                    "Async data transfer between host and device",
                    "Data structure optimization for data locality",
                    "Tuning kernel size",
                ],
            },
        },
    },
    "Enable compiler optimizations": {
        "count": 27,
        "subcategories": {
            "Function inlining": 4,
            "Enable compile time evaluation": 5,
            "Scalar Replacement": 1,
            "Loop optimization": {
                "count": 17,
                "sub_subcategories": [
                    "Loop fission",
                    "kernel fusion",
                    "Loop unrolling",
                    "Loop invariant code motion",
                ],
            },
        },
    },
    "Faster API call": {"count": 2, "subcategories": {}},
    "Introduce parallelization": {
        "count": 11,
        "subcategories": {
            "OpenMP": 2,
            "oneAPI": 1,
            "Introduce vectorization": 5,
            "pthread-based paralleism": 3,
        },
    },
    "Balancing parallel load": {
        "count": 17,
        "subcategories": {
            "Explicitly set thread numbers": 4,
            "Sort workload and schedule": 1,
            "Tune task size": 9,
            "Reduce parallelization": 3,
        },
    },
    "Improved concurrency control": {
        "count": 7,
        "subcategories": {
            "Eliminate unnecessary lock": 3,
            "Efficient lock implementation with atomics": 1,
            "Replace strong consistency with an weaker one": 1,
            "Eliminate unncessary synchronization": 1,
            "Rewrite kernel to make memory access warp-synchronous": 1,
        },
    },    "Memory management": {
        "count": 13,
        "subcategories": {
            "Avoid memory leak by memory management": 5,
            "Pre-allocation": 4,
            "Increase memory size for efficiency": 1,
            "Use efficient API": 2,
            "Reference rather than copy": 1,
        },
    },
    "fix programming error": {"count": 4, "subcategories": {}},
}

# Github scraping example 

In [7]:
def get_github_commit_info(commit_url):
    response = requests.get(commit_url)
    if response.status_code != 200:
        print("Error: Unable to fetch the webpage.")
        return

    soup = BeautifulSoup(response.content, "html.parser")

    # Get commit message
    commit_message = soup.find("div", class_="commit-desc").get_text().strip()
    #print(f"Commit message: {commit_message}")
    
    # Get code changes
    diff_elements = soup.find_all("table", class_="highlight tab-size js-file-line-container")
    for diff_element in diff_elements:
        diff_code = diff_element.get_text()
        print(diff_code)
    return commit_message

commit_url = "https://github.com/CGAL/cgal/commit/28a9cb150ae9b11f9bb37d972be990d87b05cbcf"
get_github_commit_info(commit_url)

'for facets vertices\nthis should be a lot more efficient'

In [26]:
owner= "CGAL" 
repo= "cgal"
commit_sha="28a9cb150ae9b11f9bb37d972be990d87b05cbcf"
url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}"
headers = {"Accept": "application/vnd.github+json"}

response = requests.get(url)

print(response.text)

{"sha":"28a9cb150ae9b11f9bb37d972be990d87b05cbcf","node_id":"MDY6Q29tbWl0MjQ2NzczOTI6MjhhOWNiMTUwYWU5YjExZjliYjM3ZDk3MmJlOTkwZDg3YjA1Y2JjZg==","commit":{"author":{"name":"Jane Tournois","email":"jane.tournois@geometryfactory.com","date":"2020-05-06T13:11:06Z"},"committer":{"name":"Jane Tournois","email":"jane.tournois@geometryfactory.com","date":"2020-05-06T13:11:06Z"},"message":"replace std::set<Vertex_handle> with std::array<Vertex_handle>\n\nfor facets vertices\nthis should be a lot more efficient","tree":{"sha":"df1d91c3c8dfdeedf7601243a061f31753b045b0","url":"https://api.github.com/repos/CGAL/cgal/git/trees/df1d91c3c8dfdeedf7601243a061f31753b045b0"},"url":"https://api.github.com/repos/CGAL/cgal/git/commits/28a9cb150ae9b11f9bb37d972be990d87b05cbcf","comment_count":0,"verification":{"verified":false,"reason":"unsigned","signature":null,"payload":null}},"url":"https://api.github.com/repos/CGAL/cgal/commits/28a9cb150ae9b11f9bb37d972be990d87b05cbcf","html_url":"https://github.com/CGAL/

In [9]:
false = False
null = None
res = {"sha":"28a9cb150ae9b11f9bb37d972be990d87b05cbcf","node_id":"MDY6Q29tbWl0MjQ2NzczOTI6MjhhOWNiMTUwYWU5YjExZjliYjM3ZDk3MmJlOTkwZDg3YjA1Y2JjZg==","commit":{"author":{"name":"Jane Tournois","email":"jane.tournois@geometryfactory.com","date":"2020-05-06T13:11:06Z"},"committer":{"name":"Jane Tournois","email":"jane.tournois@geometryfactory.com","date":"2020-05-06T13:11:06Z"},"message":"replace std::set<Vertex_handle> with std::array<Vertex_handle>\n\nfor facets vertices\nthis should be a lot more efficient","tree":{"sha":"df1d91c3c8dfdeedf7601243a061f31753b045b0","url":"https://api.github.com/repos/CGAL/cgal/git/trees/df1d91c3c8dfdeedf7601243a061f31753b045b0"},"url":"https://api.github.com/repos/CGAL/cgal/git/commits/28a9cb150ae9b11f9bb37d972be990d87b05cbcf","comment_count":0,"verification":{"verified":false,"reason":"unsigned","signature":null,"payload":null}},"url":"https://api.github.com/repos/CGAL/cgal/commits/28a9cb150ae9b11f9bb37d972be990d87b05cbcf","html_url":"https://github.com/CGAL/cgal/commit/28a9cb150ae9b11f9bb37d972be990d87b05cbcf","comments_url":"https://api.github.com/repos/CGAL/cgal/commits/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/comments","author":{"login":"janetournois","id":11311305,"node_id":"MDQ6VXNlcjExMzExMzA1","avatar_url":"https://avatars.githubusercontent.com/u/11311305?v=4","gravatar_id":"","url":"https://api.github.com/users/janetournois","html_url":"https://github.com/janetournois","followers_url":"https://api.github.com/users/janetournois/followers","following_url":"https://api.github.com/users/janetournois/following{/other_user}","gists_url":"https://api.github.com/users/janetournois/gists{/gist_id}","starred_url":"https://api.github.com/users/janetournois/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/janetournois/subscriptions","organizations_url":"https://api.github.com/users/janetournois/orgs","repos_url":"https://api.github.com/users/janetournois/repos","events_url":"https://api.github.com/users/janetournois/events{/privacy}","received_events_url":"https://api.github.com/users/janetournois/received_events","type":"User","site_admin":false},"committer":{"login":"janetournois","id":11311305,"node_id":"MDQ6VXNlcjExMzExMzA1","avatar_url":"https://avatars.githubusercontent.com/u/11311305?v=4","gravatar_id":"","url":"https://api.github.com/users/janetournois","html_url":"https://github.com/janetournois","followers_url":"https://api.github.com/users/janetournois/followers","following_url":"https://api.github.com/users/janetournois/following{/other_user}","gists_url":"https://api.github.com/users/janetournois/gists{/gist_id}","starred_url":"https://api.github.com/users/janetournois/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/janetournois/subscriptions","organizations_url":"https://api.github.com/users/janetournois/orgs","repos_url":"https://api.github.com/users/janetournois/repos","events_url":"https://api.github.com/users/janetournois/events{/privacy}","received_events_url":"https://api.github.com/users/janetournois/received_events","type":"User","site_admin":false},"parents":[{"sha":"0a43f88f5d600b29ec26b3a46286fd20bc0759ab","url":"https://api.github.com/repos/CGAL/cgal/commits/0a43f88f5d600b29ec26b3a46286fd20bc0759ab","html_url":"https://github.com/CGAL/cgal/commit/0a43f88f5d600b29ec26b3a46286fd20bc0759ab"}],"stats":{"total":151,"additions":93,"deletions":58},"files":[{"sha":"707553f88ae9d8e62344fbf08d6d9edb38b59630","filename":"Mesh_3/include/CGAL/Mesh_3/tet_soup_to_c3t3.h","status":"modified","additions":88,"deletions":55,"changes":143,"blob_url":"https://github.com/CGAL/cgal/blob/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/Mesh_3%2Finclude%2FCGAL%2FMesh_3%2Ftet_soup_to_c3t3.h","raw_url":"https://github.com/CGAL/cgal/raw/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/Mesh_3%2Finclude%2FCGAL%2FMesh_3%2Ftet_soup_to_c3t3.h","contents_url":"https://api.github.com/repos/CGAL/cgal/contents/Mesh_3%2Finclude%2FCGAL%2FMesh_3%2Ftet_soup_to_c3t3.h?ref=28a9cb150ae9b11f9bb37d972be990d87b05cbcf","patch":"@@ -29,6 +29,17 @@\n \n namespace CGAL\n {\n+\n+template<typename Vh>\n+std::array<Vh, 3> make_ordered_vertex_array(const Vh vh0, const Vh vh1, const Vh vh2)\n+{\n+  std::array<Vh, 3> ft = { {vh0, vh1, vh2} };\n+  if (ft[1] < ft[0]) std::swap(ft[0], ft[1]);\n+  if (ft[2] < ft[1]) std::swap(ft[1], ft[2]);\n+  if (ft[1] < ft[0]) std::swap(ft[0], ft[1]);\n+  return ft;\n+}\n+\n template<class Tr>\n void build_vertices(Tr& tr,\n                     const std::vector<typename Tr::Point>& points,\n@@ -49,22 +60,22 @@ void build_vertices(Tr& tr,\n }\n \n template<class Tr>\n-void add_facet_to_incident_cells_map(const typename Tr::Cell_handle c, int i,\n-    boost::unordered_map<std::set<typename Tr::Vertex_handle>,\n-                         std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map)\n+bool add_facet_to_incident_cells_map(const typename Tr::Cell_handle c, int i,\n+    boost::unordered_map<std::array<typename Tr::Vertex_handle, 3>,\n+                         std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map,\n+    const bool verbose)\n {\n   typedef typename Tr::Vertex_handle                                Vertex_handle;\n   typedef typename Tr::Cell_handle                                  Cell_handle;\n-  typedef std::set<Vertex_handle>                                   Facet;\n+  typedef std::array<Vertex_handle, 3>                              Facet_vvv;\n   typedef std::pair<Cell_handle, int>                               Incident_cell;\n-  typedef boost::unordered_map<Facet, std::vector<Incident_cell> >  Incident_cells_map;\n+  typedef boost::unordered_map<Facet_vvv, std::vector<Incident_cell> > Incident_cells_map;\n \n   // the opposite vertex of f in c is i\n-  Facet f;\n-  f.insert(c->vertex((i + 1) % 4));\n-  f.insert(c->vertex((i + 2) % 4));\n-  f.insert(c->vertex((i + 3) % 4));\n-  CGAL_precondition(f.size() == 3);\n+  Facet_vvv f = make_ordered_vertex_array(c->vertex((i + 1) % 4),\n+                                          c->vertex((i + 2) % 4),\n+                                          c->vertex((i + 3) % 4));\n+  CGAL_precondition(f[0] != f[1] && f[1] != f[2]);\n \n   Incident_cell e = std::make_pair(c, i);\n   std::vector<Incident_cell> vec;\n@@ -74,33 +85,25 @@ void add_facet_to_incident_cells_map(const typename Tr::Cell_handle c, int i,\n   if(!is_insert_successful.second) // the entry already exists in the map\n   {\n     // a facet must have exactly two incident cells\n-//    if (is_insert_successful.first->second.size() != 1)\n-//    {\n-//      typename Tr::Geom_traits::Construct_point_3 point\n-//        = typename Tr::Geom_traits().construct_point_3_object();\n-//      for (auto fi : incident_cells_map.at(f))\n-//      {\n-//        std::cout << point(fi.first->vertex((fi.second + 1) % 4)->point())\n-//           << \" \" << point(fi.first->vertex((fi.second + 2) % 4)->point())\n-//           << \" \" << point(fi.first->vertex((fi.second + 3) % 4)->point()) << std::endl;\n-//      }\n-//      std::cout << \"finite facet : \" << std::endl;\n-//      std::cout << point(c->vertex(1)->point())\n-//         << \" \" << point(c->vertex(2)->point())\n-//         << \" \" << point(c->vertex(3)->point()) << std::endl;\n-//    }\n-    CGAL_assertion(is_insert_successful.first->second.size() == 1);\n+    if (is_insert_successful.first->second.size() != 1)\n+    {\n+      if(verbose)\n+        std::cout << \"Error in add_facet_to_incident_cells_map\" << std::endl;\n+      return false;\n+    }\n     is_insert_successful.first->second.push_back(e);\n   }\n+  return true;\n }\n \n template<class Tr>\n-void build_finite_cells(Tr& tr,\n-                        const std::vector<std::array<int,5> >& finite_cells,\n-                        const std::vector<typename Tr::Vertex_handle>& vertex_handle_vector,\n-                        boost::unordered_map<std::set<typename Tr::Vertex_handle>,\n+bool build_finite_cells(Tr& tr,\n+    const std::vector<std::array<int,5> >& finite_cells,\n+    const std::vector<typename Tr::Vertex_handle>& vertex_handle_vector,\n+    boost::unordered_map<std::array<typename Tr::Vertex_handle, 3>,\n                         std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map,\n-                        const std::map<std::array<int,3>, int>& border_facets)\n+    const std::map<std::array<int,3>, int>& border_facets,\n+    const bool verbose)\n {\n   typedef std::array<int, 5>              Tet_with_ref; // 4 ids + 1 reference\n \n@@ -145,7 +148,8 @@ void build_finite_cells(Tr& tr,\n     // build the map used for adjacency later\n     for(int j=0; j<4; ++j)\n     {\n-      add_facet_to_incident_cells_map<Tr>(c, j, incident_cells_map);\n+      if(!add_facet_to_incident_cells_map<Tr>(c, j, incident_cells_map, verbose))\n+        return false;\n       if(border_facets.size() != 0)\n       {\n         std::array<int,3> facet;\n@@ -184,33 +188,36 @@ void build_finite_cells(Tr& tr,\n       }\n     }\n   }\n+  return true;\n }\n \n template<class Tr>\n-void add_infinite_facets_to_incident_cells_map(typename Tr::Cell_handle c,\n+bool add_infinite_facets_to_incident_cells_map(typename Tr::Cell_handle c,\n      int inf_vert_pos,\n-     boost::unordered_map<std::set<typename Tr::Vertex_handle>,\n-                          std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map)\n+     boost::unordered_map<std::array<typename Tr::Vertex_handle, 3>,\n+                          std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map,\n+     const bool verbose)\n {\n   int l = (inf_vert_pos + 1) % 4;\n-  add_facet_to_incident_cells_map<Tr>(c, l, incident_cells_map);\n+  bool b1 = add_facet_to_incident_cells_map<Tr>(c, l, incident_cells_map, verbose);\n   l = (inf_vert_pos + 2) % 4;\n-  add_facet_to_incident_cells_map<Tr>(c, l, incident_cells_map);\n+  bool b2 = add_facet_to_incident_cells_map<Tr>(c, l, incident_cells_map, verbose);\n   l = (inf_vert_pos + 3) % 4;\n-  add_facet_to_incident_cells_map<Tr>(c, l, incident_cells_map);\n+  bool b3 = add_facet_to_incident_cells_map<Tr>(c, l, incident_cells_map, verbose);\n+  return b1 && b2 && b3;\n }\n \n template<class Tr>\n-void build_infinite_cells(Tr& tr,\n-                          boost::unordered_map<std::set<typename Tr::Vertex_handle>,\n-                          std::vector<std::pair<typename Tr::Cell_handle,\n-                          int> > >& incident_cells_map)\n+bool build_infinite_cells(Tr& tr,\n+  boost::unordered_map<std::array<typename Tr::Vertex_handle, 3>,\n+                       std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map,\n+  const bool verbose)\n {\n   typedef typename Tr::Vertex_handle                               Vertex_handle;\n   typedef typename Tr::Cell_handle                                 Cell_handle;\n-  typedef std::set<Vertex_handle>                                  Facet;\n+  typedef std::array<Vertex_handle, 3>                             Facet_vvv;\n   typedef std::pair<Cell_handle, int>                              Incident_cell;\n-  typedef boost::unordered_map<Facet, std::vector<Incident_cell> > Incident_cells_map;\n+  typedef boost::unordered_map<Facet_vvv, std::vector<Incident_cell> > Incident_cells_map;\n \n   std::vector<Cell_handle> infinite_cells;\n \n@@ -251,22 +258,45 @@ void build_infinite_cells(Tr& tr,\n     CGAL_assertion(it->second.size() == 2);\n   }\n \n+#ifdef CGAL_TET_SOUP_TO_C3T3_DEBUG\n+  for (auto icit : incident_cells_map)\n+    CGAL_assertion(icit.second.size() == 2);\n+\n+  std::map<Facet_vvv, int> facets;\n+  for (const Cell_handle c : infinite_cells)\n+  {\n+    for (int i = 1; i < 4; ++i)\n+    {\n+      std::array<Vertex_handle, 3> vs = make_ordered_vertex_array(c->vertex((i + 1) % 4),\n+        c->vertex((i + 2) % 4),\n+        c->vertex((i + 3) % 4));\n+      if (facets.find(vs) == facets.end())\n+        facets.insert(std::make_pair(vs, 1));\n+      else\n+        facets[vs]++;\n+    }\n+  }\n+  for (auto fp : facets)\n+    CGAL_assertion(fp.second == 2);\n+#endif\n+\n   // add the facets to the incident cells map\n   for (const Cell_handle c : infinite_cells)\n-    add_infinite_facets_to_incident_cells_map<Tr>(c, 0, incident_cells_map);\n+    if(!add_infinite_facets_to_incident_cells_map<Tr>(c, 0, incident_cells_map, verbose))\n+      return false;\n+\n+  return true;\n }\n \n template<class Tr>\n bool assign_neighbors(Tr& tr,\n-                      const boost::unordered_map<std::set<typename Tr::Vertex_handle>,\n-                      std::vector<std::pair<typename Tr::Cell_handle,\n-                      int> > >& incident_cells_map)\n+  const boost::unordered_map<std::array<typename Tr::Vertex_handle, 3>,\n+                             std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map)\n {\n-  typedef typename Tr::Vertex_handle                                 Vertex_handle;\n   typedef typename Tr::Cell_handle                                   Cell_handle;\n-  typedef std::set<Vertex_handle>                                    Facet;\n   typedef std::pair<Cell_handle, int>                                Incident_cell;\n-  typedef boost::unordered_map<Facet, std::vector<Incident_cell> >   Incident_cells_map;\n+  typedef boost::unordered_map<std::array<typename Tr::Vertex_handle, 3>,\n+                               std::vector<Incident_cell> >          Incident_cells_map;\n \n   typename Incident_cells_map::const_iterator icit = incident_cells_map.begin();\n   for(; icit!=incident_cells_map.end(); ++icit)\n@@ -295,11 +325,11 @@ bool build_triangulation(Tr& tr,\n {\n   typedef typename Tr::Vertex_handle            Vertex_handle;\n   typedef typename Tr::Cell_handle              Cell_handle;\n-  typedef std::set<Vertex_handle>               Facet;\n+  typedef std::array<Vertex_handle, 3>          Facet_vvv;\n \n   // associate to a face the two (at most) incident tets and the id of the face in the cell\n   typedef std::pair<Cell_handle, int>                   Incident_cell;\n-  typedef boost::unordered_map<Facet, std::vector<Incident_cell> >  Incident_cells_map;\n+  typedef boost::unordered_map<Facet_vvv, std::vector<Incident_cell> >  Incident_cells_map;\n \n   Incident_cells_map incident_cells_map;\n   vertex_handle_vector.resize(points.size() + 1); // id to vertex_handle\n@@ -322,8 +352,11 @@ bool build_triangulation(Tr& tr,\n   }\n   if (!finite_cells.empty())\n   {\n-    build_finite_cells<Tr>(tr, finite_cells, vertex_handle_vector, incident_cells_map, border_facets);\n-    build_infinite_cells<Tr>(tr, incident_cells_map);\n+    if(!build_finite_cells<Tr>(tr, finite_cells, vertex_handle_vector, incident_cells_map,\n+                           border_facets, verbose))\n+      return false;\n+    if(!build_infinite_cells<Tr>(tr, incident_cells_map, verbose))\n+      return false;\n     tr.tds().set_dimension(3);\n     if (!assign_neighbors<Tr>(tr, incident_cells_map))\n       return false;"},{"sha":"e1e9044f0b6b6cd37b935f2b319d28cfb7d12b0f","filename":"Polyhedron/demo/Polyhedron/Plugins/IO/VTK_io_plugin.cpp","status":"modified","additions":5,"deletions":3,"changes":8,"blob_url":"https://github.com/CGAL/cgal/blob/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/Polyhedron%2Fdemo%2FPolyhedron%2FPlugins%2FIO%2FVTK_io_plugin.cpp","raw_url":"https://github.com/CGAL/cgal/raw/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/Polyhedron%2Fdemo%2FPolyhedron%2FPlugins%2FIO%2FVTK_io_plugin.cpp","contents_url":"https://api.github.com/repos/CGAL/cgal/contents/Polyhedron%2Fdemo%2FPolyhedron%2FPlugins%2FIO%2FVTK_io_plugin.cpp?ref=28a9cb150ae9b11f9bb37d972be990d87b05cbcf","patch":"@@ -503,8 +503,8 @@ class Polyhedron_demo_vtk_plugin :\n \n     if (is_c3t3)\n     {\n-      typedef boost::array<int, 3> Facet; // 3 = id\n-      typedef boost::array<int, 5> Tet_with_ref; // first 4 = id, fifth = reference\n+      typedef std::array<int, 3> Facet; // 3 = id\n+      typedef std::array<int, 5> Tet_with_ref; // first 4 = id, fifth = reference\n       Scene_c3t3_item* c3t3_item = new Scene_c3t3_item();\n       c3t3_item->set_valid(false);\n       //build a triangulation from data:\n@@ -547,7 +547,9 @@ class Polyhedron_demo_vtk_plugin :\n           std::swap(finite_cells[i][1], finite_cells[i][3]);\n         }\n       }\n-      CGAL::build_triangulation<Tr, true>(c3t3_item->c3t3().triangulation(), points, finite_cells, border_facets);\n+      std::vector<typename Tr::Vertex_handle> new_vertices;\n+      CGAL::build_triangulation<Tr, true>(c3t3_item->c3t3().triangulation(),\n+        points, finite_cells, border_facets, new_vertices);\n \n       for( C3t3::Triangulation::Finite_cells_iterator\n            cit = c3t3_item->c3t3().triangulation().finite_cells_begin();"}]}
print("in res:")
for i in res:   
    print("\t",i)
print("in commit")
for i in res["commit"]:   
    print("\t",i)
print("in files")
for i in res["files"]:   
    print("\t",i)

in res:
	 sha
	 node_id
	 commit
	 url
	 html_url
	 comments_url
	 author
	 committer
	 parents
	 stats
	 files
in commit
	 author
	 committer
	 message
	 tree
	 url
	 comment_count
	 verification
in files
	 {'sha': '707553f88ae9d8e62344fbf08d6d9edb38b59630', 'filename': 'Mesh_3/include/CGAL/Mesh_3/tet_soup_to_c3t3.h', 'status': 'modified', 'additions': 88, 'deletions': 55, 'changes': 143, 'blob_url': 'https://github.com/CGAL/cgal/blob/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/Mesh_3%2Finclude%2FCGAL%2FMesh_3%2Ftet_soup_to_c3t3.h', 'raw_url': 'https://github.com/CGAL/cgal/raw/28a9cb150ae9b11f9bb37d972be990d87b05cbcf/Mesh_3%2Finclude%2FCGAL%2FMesh_3%2Ftet_soup_to_c3t3.h', 'contents_url': 'https://api.github.com/repos/CGAL/cgal/contents/Mesh_3%2Finclude%2FCGAL%2FMesh_3%2Ftet_soup_to_c3t3.h?ref=28a9cb150ae9b11f9bb37d972be990d87b05cbcf', 'patch': '@@ -29,6 +29,17 @@\n \n namespace CGAL\n {\n+\n+template<typename Vh>\n+std::array<Vh, 3> make_ordered_vertex_array(const Vh vh0, const Vh vh1, c

In [10]:
files= res["files"]   
print("int the files")
for f in files:
    for i in f:
        print("\t",i)    
    break

int the files
	 sha
	 filename
	 status
	 additions
	 deletions
	 changes
	 blob_url
	 raw_url
	 contents_url
	 patch


In [11]:
#geting the whole old version is too long and not feasible for LLM clasification
# Replace these with the appropriate values
file_path = "Mesh_3/include/CGAL/Mesh_3/tet_soup_to_c3t3.h"

# Fetch the parent commit SHA
api_url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}"
headers = {"Accept": "application/vnd.github+json"}

response = requests.get(api_url, headers=headers)

if response.status_code == 200:
    commit_data = response.json()
    if commit_data["parents"]:
        parent_sha = commit_data["parents"][0]["sha"]

        # Fetch the file from the parent commit
        contents_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{file_path}?ref={parent_sha}"
        response = requests.get(contents_url, headers=headers)

        if response.status_code == 200:
            file_data = response.json()
            file_content = requests.get(file_data["download_url"]).text
            print("File content before the commit:")
            print(file_content)
        else:
            print(f"Error: Unable to fetch file from the parent commit. Status code: {response.status_code}")
    else:
        print("Error: The specified commit is the initial commit and has no parent.")
else:
    print(f"Error: Unable to fetch commit information. Status code: {response.status_code}")


File content before the commit:
// Copyright (c) 2009 INRIA Sophia-Antipolis (France).
// All rights reserved.
//
// This file is part of CGAL (www.cgal.org).
//
// $URL$
// $Id$
// SPDX-License-Identifier: GPL-3.0-or-later OR LicenseRef-Commercial
//
//
// Author(s)     : Mael Rouxel-Labbé, Maxime Gimeno
//
//******************************************************************************
//
//******************************************************************************

#ifndef CGAL_MESH_3_TET_SOUP_TO_C3T3_H
#define CGAL_MESH_3_TET_SOUP_TO_C3T3_H

#include <CGAL/license/Mesh_3.h>

#include <CGAL/disable_warnings.h>
#include <CGAL/assertions.h>
#include <CGAL/IO/File_medit.h>

#include <array>
#include <boost/unordered_map.hpp>


namespace CGAL
{
template<class Tr>
void build_vertices(Tr& tr,
                    const std::vector<typename Tr::Point>& points,
                    std::vector<typename Tr::Vertex_handle>& vertex_handle_vector)
{
  typedef typename Tr::Vertex_handle        

this gives the whole file to big

In [12]:
files= res["files"]   
print("int the files")
for f in files:
    for i in f:
        print("\t",i)    
    break
 
print("int the files")
for f in files:    
    print("\t",f["patch"])    
    break

int the files
	 sha
	 filename
	 status
	 additions
	 deletions
	 changes
	 blob_url
	 raw_url
	 contents_url
	 patch
int the files
	 @@ -29,6 +29,17 @@
 
 namespace CGAL
 {
+
+template<typename Vh>
+std::array<Vh, 3> make_ordered_vertex_array(const Vh vh0, const Vh vh1, const Vh vh2)
+{
+  std::array<Vh, 3> ft = { {vh0, vh1, vh2} };
+  if (ft[1] < ft[0]) std::swap(ft[0], ft[1]);
+  if (ft[2] < ft[1]) std::swap(ft[1], ft[2]);
+  if (ft[1] < ft[0]) std::swap(ft[0], ft[1]);
+  return ft;
+}
+
 template<class Tr>
 void build_vertices(Tr& tr,
                     const std::vector<typename Tr::Point>& points,
@@ -49,22 +60,22 @@ void build_vertices(Tr& tr,
 }
 
 template<class Tr>
-void add_facet_to_incident_cells_map(const typename Tr::Cell_handle c, int i,
-    boost::unordered_map<std::set<typename Tr::Vertex_handle>,
-                         std::vector<std::pair<typename Tr::Cell_handle, int> > >& incident_cells_map)
+bool add_facet_to_incident_cells_map(const typename Tr::Cell_handl

In [13]:
Old_versions=[]

def parse_patch_no_lines(patch):
    lines = []

    for line in patch.splitlines():
        if line.startswith('-'):
            lines.append(line[1:])
        elif line.startswith(' '):
            lines.append(line[1:])

    return '\n'.join(lines)

def parse_patch(patch):
    lines = []
    old_line_number = None
    new_line_number = None

    for line in patch.splitlines():
        if line.startswith('@@'):
            old_line_number, new_line_number = map(int, re.findall(r'\d+', line)[:2])
        elif line.startswith('-'):
            lines.append(f"{old_line_number}: {line[1:]}")
            old_line_number += 1
        elif line.startswith('+'):
            new_line_number += 1
        elif line.startswith(' '):
            lines.append(f"{old_line_number}: {line[1:]}")
            old_line_number += 1
            new_line_number += 1

    return '\n'.join(lines)


# Get the commit information
api_url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}"
headers = {"Accept": "application/vnd.github+json"}

commit_response = requests.get(api_url, headers=headers)

if commit_response.status_code == 200:
    commit_data = commit_response.json()
    files = commit_data["files"]

    for file in files:
        print(f"File: {file['filename']}")
        
        patch = file.get("patch")
        if patch:
            old_version_changes = parse_patch(patch)
            print("Old version of the changed parts:")
            Old_versions.append( old_version_changes )
        else:
            print("No changes detected in this file.")
        
        print("\n")
else:
    print(f"Error: Unable to fetch commit information. Status code: {commit_response.status_code}")

File: Mesh_3/include/CGAL/Mesh_3/tet_soup_to_c3t3.h
Old version of the changed parts:


File: Polyhedron/demo/Polyhedron/Plugins/IO/VTK_io_plugin.cpp
Old version of the changed parts:




In [14]:
def remove_unimportant_spaces(code: str) -> str:
    # Remove spaces and tabs around operators and punctuation
    code = re.sub(r'\s*([\+\-\*/%=&\|!<>;,\(\)\{\}\[\]])\s*', r' \1 ', code)

    # Remove extra spaces and tabs at the beginning and end of each line
    code = '\n'.join(line.strip() for line in code.split('\n'))

    # Replace consecutive spaces with a single space
    code = re.sub(r' +', ' ', code)

    return code

In [15]:
print(f"first len : {len(Old_versions[0])}")
print(f"second len : { len( remove_unimportant_spaces ( Old_versions[0]))  }")


first len : 6711
second len : 6209


In [16]:
first_code_to_send = remove_unimportant_spaces ( Old_versions[0])

# Open ai api example

In [17]:
catagories = ""
for i in RQ1_dict:
    catagories += i + ", "
#print(catagories)

system_promt = f"You are a highly intelligent software high performance engineering assistant" + f"when given a piece  of code you will classify the performance problems in the code in to one of these catagories: {catagories}."

system_promt

'You are a highly intelligent software high performance engineering assistantwhen given a piece  of code you will classify the performance problems in the code in to one of these catagories: Inefficient coding for target micro-architecure, Missing parallelism, Parallelization overhead/inefficiency, Inefficient Concurrency control and synchronization, Unnecessary process communication, Inefficient algorithm /data-structure and their implementation, Inefficient memory management, I/O inefficiency, Unintentional Programming logic error, Inefficiency due to new compiler version, .'

In [18]:
openai.api_key = openai_key

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = [
      {"role": "system", "content": system_promt},
      {"role": "user", "content": f"what is the fallowing patch catagories to ?\m {get_github_commit_info(commit_url)}  ?  "} ,       
    ],   
  
)
print(response['choices'][0]['message']["content"])


Based on the limited information provided, I cannot determine the patch categories. However, the statement "this should be a lot more efficient" suggests that the patch is likely addressing inefficiencies in the code. It is possible that the patch is trying to optimize the processing of facets and vertices by improving the data structures or implementing more efficient algorithms. Without more context or code examples, it is difficult to provide a more specific categorization of the patch.


In [19]:
git_links=  []
with open ("Data/RQ1/links.txt","r") as f:
    for i in f :
        print(i.replace('\n', ''))   
        git_links.append(i.replace('\n', ''))    

https://github.com/CGAL/cgal/commit/28a9cb150ae9b11f9bb37d972be990d87b05cbcf
https://github.com/CGAL/cgal/commit/b0379f54bdff8236dee3a429b235167700ba23c0
https://github.com/mlpack/mlpack/commit/723dea88435686b72359793ace767d0b409635af
https://github.com/CGAL/cgal/commit/8855eb54c305ffeaec32cb6bc49bfb0a2e465cc2
https://github.com/lammps/lammps/commit/6c5edf6c709ef1fc9c914ec8049fa41a7fb3a49a
https://github.com/gromacs/gromacs/commit/001b6e76700a2228a60d28223f7fb37ed5b7d3be
https://github.com/xianyi/OpenBLAS/commit/45fdf951b64aa9145996727ecda901f00a2eda3c
https://github.com/cp2k/cp2k/commit/7b34ac6ca470a6a5a2d71d2027d577b08f0dac92
https://github.com/FFTW/fftw3/commit/d0b93533d99e69f85e2aaf759989f652311206ac
https://github.com/gromacs/gromacs/commit/95cc614f64ac211696da00204923e387a735a490
https://github.com/FFTW/fftw3/commit/6076339a342b12b0d0cfd9f6d967bfa9fbf6b1b2
https://github.com/xianyi/OpenBLAS/commit/1b0b4349a11f8de40037d9bddf9ddb9b094cdd2c
https://github.com/xianyi/OpenBLAS/commit/

In [37]:
path=".\\Data\\RQ1\\RQ1.xlsx"
listt=[]
actualList=[]
ownerList=[]
ProjectList=[]
HashList=[]
df = pd.read_excel(path)
#print(df.iloc[:,17])
listt.append(df.iloc[:,17])
#print(listt[0])
for i in listt[0]:
    actualList.append(i)
actualList.pop()
for i in actualList:
    text=i[19:]
    splittedVer=text.split('/')
    ownerList.append(splittedVer[0])
    ProjectList.append(splittedVer[1])
    HashList.append(splittedVer[3])
    


In [40]:
headers = {    
    "Authorization": f"token {github_key}"
}

In [52]:
all_git_responses={}
false = False
null = None
#pickled this list
"""
for owner , repo , commit_sha , link in zip(ownerList,ProjectList,HashList,actualList ):    
    url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}"
    response = requests.get(url, headers=headers)
    print(response.json() )
    all_git_responses[link] = dict(response.json())
with open("./pickles/git_respos.pkl", "wb") as f:
    pickle.dump(all_git_responses, f)
"""
with open("./pickles/git_respos.pkl", "rb") as f:
    all_git_responses = pickle.load(f) 

In [69]:
#all_files_in_a_comit

In [73]:
all_files_in_a_comit={}
for repo in all_git_responses:
    all_files_in_a_comit[repo]=all_git_responses[repo]["files"]

all_patches_in_all_files_in_a_comit ={}
for repo in all_files_in_a_comit:
    all_patches_in_all_files_in_a_comit[repo]=[]
    for file in all_files_in_a_comit[repo]:
        all_patches_in_all_files_in_a_comit[repo].append( remove_unimportant_spaces( parse_patch( file["patch"]) ))


In [75]:
catagories = ""
for i in RQ1_dict:
    catagories += i + ", "
#print(catagories)

system_promt = f"You are a highly intelligent software high performance engineering assistant" + f"when given a piece  of code you will classify the performance problems in the code in to one of these catagories: {catagories}."

system_promt

'You are a highly intelligent software high performance engineering assistantwhen given a piece  of code you will classify the performance problems in the code in to one of these catagories: Inefficient coding for target micro-architecure, Missing parallelism, Parallelization overhead/inefficiency, Inefficient Concurrency control and synchronization, Unnecessary process communication, Inefficient algorithm /data-structure and their implementation, Inefficient memory management, I/O inefficiency, Unintentional Programming logic error, Inefficiency due to new compiler version, .'

In [81]:
all_responses_in_all_files_in_a_comit ={}
""" #takes 200 minuts to run !!!!!!!
for repo in all_patches_in_all_files_in_a_comit:  
    all_responses_in_all_files_in_a_comit[repo]=[]  
    for patch  in all_patches_in_all_files_in_a_comit[repo]:
        #print(patch)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages = [
                {"role": "system", "content": system_promt},
                {"role": "user", "content": f"what is the fallowing patch catagories to ?\m {get_github_commit_info(commit_url)}  ?  "} ,       
                ],  
        )
        all_responses_in_all_files_in_a_comit[repo].append(response['choices'][0]['message']["content"])
        time.sleep(25) # to avoid openai api rate limit
print(all_responses_in_all_files_in_a_comit)
with open("./pickles/GPT3_5_respos.pkl", "wb") as f:
    pickle.dump(all_responses_in_all_files_in_a_comit, f)
"""
with open("./pickles/GPT3_5_respos.pkl", "rb") as f:
    all_responses_in_all_files_in_a_comit = pickle.load(f)

{'https://github.com/CGAL/cgal/commit/28a9cb150ae9b11f9bb37d972be990d87b05cbcf': ['It seems like you have provided incomplete information about the context of the patch. However, based on the provided information, it can be classified under "Inefficient Coding for Target Micro-Architecture", as the patch is intended to improve the efficiency of the code by optimizing its performance for the underlying hardware architecture. Specifically, the patch aims to optimize the computation of facet vertices by using a more efficient algorithm or data structure, resulting in faster execution and better resource utilization.', 'The given patch statement does not clearly specify any performance problems or categories. However, it seems like the patch is trying to optimize the code for efficiency by improving the operation performed on facets vertices. Without knowing the specifics of the code and the context, it is not possible to classify the performance problems.'], 'https://github.com/CGAL/cgal/

In [86]:
for i in all_responses_in_all_files_in_a_comit:
    print(f"for the commit {i} the ais classification is : ")
    for j in all_responses_in_all_files_in_a_comit[i]:
        print(f"-\t\t{j}")
    print()

for the commit https://github.com/CGAL/cgal/commit/28a9cb150ae9b11f9bb37d972be990d87b05cbcf the ais classification is : 
-		It seems like you have provided incomplete information about the context of the patch. However, based on the provided information, it can be classified under "Inefficient Coding for Target Micro-Architecture", as the patch is intended to improve the efficiency of the code by optimizing its performance for the underlying hardware architecture. Specifically, the patch aims to optimize the computation of facet vertices by using a more efficient algorithm or data structure, resulting in faster execution and better resource utilization.
-		The given patch statement does not clearly specify any performance problems or categories. However, it seems like the patch is trying to optimize the code for efficiency by improving the operation performed on facets vertices. Without knowing the specifics of the code and the context, it is not possible to classify the performance pr